This notebook takes '2.SNeCOMasterCatManualClean.txt' (see prepwork below) and converts it into a data frame and performs some basic statistics on the supernova sample.

Honestly this code isn't doing a whole lot right now but keep it available to potentially run more simple statistics.

*****  PREPWORK  *****
After looking at the galaxy images from #2. We find we need to remove one SN that isnt within the region covered by the PHANGS-ALMA footprint. We remove this (SN2003gd from NGC0628) and also the 6 candidate SNe, the one duplicate entry, and the LRN, and begin to perform some rudimentary stats in small tables.

Before we run this code we do the following prep:

We take the 2.SNeCOMasterCat.txt file and manually remove the following SNe:

Previously Removed
AT2020nqq   (LRN) 
PTSS-19clju (Duplicate)

Recently commented out
AT2018eoq   (Candidate)
AT2019pck   (Candidate)
AT2019npi   (Candidate)
AT2019npd   (Candidate)
AT2020hol   (Candidate)
AT2020juh   (Candidate)
AT2020cwh   (Candidate)
SN2003gd    (Out of Map ngc0628)
SN2005af    (Out of Map ngc4945)
ngc5236,PS  (Not in Map & Also not good data on this one)

*also shortened type IIn, IIb to II and type Ia-91bg to Ia, etc.

Then we save this as 2.SNeCOMasterCatManualClean.txt

We turn this file into a dataframe and use to make basic tables of stats.
**********************

In [4]:
import numpy as np
from matplotlib import pyplot as plt
import astropy.io.fits as pyfits
from astropy.table import Table
from astropy.wcs import WCS
from astropy.io import ascii

In [7]:
# Turn file into dataStructure and clean

dataFile = '../Data/2.SNeCOMasterCatManualClean.txt'
galaxy, SNname, SNtype, ra, dec, xVal, yVal, co_21_int, co_21_err, co_21_ew, beamsize  = np.genfromtxt(dataFile,  dtype=str, unpack=True, delimiter = ',')

SNname = [x[2:-2] for x in SNname]
SNtype = [x.strip() for x in SNtype]
ra     = [float(x) for x in ra]
ra     = [round(x,4) for x in ra]
dec    = [float(x) for x in dec]
dec    = [round(x,4) for x in dec]

data = Table({'Galaxy'    : galaxy,
              'SN Name'  : SNname,
              'SN Type'  : SNtype,
              'ra'       : ra,
              'Dec'      : dec,
              'xCoord'   : xVal,
              'yCoord'   : yVal,
              'co21int'  : co_21_int,
              'co21err'  : co_21_err,
              'co21ew'   : co_21_ew,
              'beamsize' : beamsize,
             },
             names=['Galaxy', 'SN Name','SN Type','ra','Dec','xCoord', 'yCoord', 'co21int', 'co21err', "co21ew", 'beamsize'])

data

Galaxy,SN Name,SN Type,ra,Dec,xCoord,yCoord,co21int,co21err,co21ew,beamsize
str18,str7,str12,float64,float64,str18,str18,str18,str18,str18,str18
circinus,1996,II,213.2918,-65.3457,86.56761411031889,84.63433198287285,186.84804,2.137819,34.56537,0.002032163908078
ngc0253,194,I,11.8783,-25.2934,296.9010534051223,236.26662905273994,219.30367,1.465811,45.862267,0.00232543637022
ngc0300,2019q,II,13.7399,-37.6444,124.97990204934794,202.45166439134732,-0.41793606,0.4800788,nan,0.002271061982879
ngc0628,2013,II,24.2007,15.7586,333.46022572413716,356.77980642079694,5.7713037,1.9949342,nan,0.0003114595189268
ngc0628,2019k,II,24.2068,15.7795,227.48504797816918,732.1448775793053,0.2809551,1.2958647,nan,0.0003114595189268
ngc1068,2018i,II,40.672,-0.0088,41.400000046852824,53.87249997022789,127.77749,0.6721418,32.0753,0.002414131938401
ngc1087,199,II,41.6115,-0.4988,161.4198245203906,287.6499560537321,11.513839,1.1399242,5.175495,0.0004451676280474
ngc1097,1999,II,41.5866,-30.3184,245.2470996170838,7.223039949207134,0.0,0.7541714,nan,0.0004711325009833
ngc1097,1992,II,41.5792,-30.2756,291.5099030167368,315.426803230518,296.15988,2.0283754,45.82589,0.0004711325009833


In [8]:
counter, IICounter, IaCounter, IbcCounter, unknownCounter = 0,0,0,0,0

for i in range(len(SNtype)):
    counter = counter + 1
    if(len(SNtype[i]) == 1 or len(SNtype[i]) > 5):
        unknownCounter = unknownCounter + 1   
    elif SNtype[i][1] == "I":
        IICounter = IICounter + 1 
    elif SNtype[i][1] == "b" or SNtype[i][1] == "c":
        IbcCounter = IbcCounter + 1 
    elif SNtype[i][1] == "a":
        IaCounter = IaCounter + 1 
    else:
        print("something fucked up")
            
print(counter, IICounter, IaCounter, IbcCounter, unknownCounter)

knownCounter = counter - unknownCounter

percentII = np.str(np.int(np.round(100*(IICounter/knownCounter)))) + "%"
percentIa = np.str(np.int(np.round(100*(IaCounter/knownCounter)))) + "%"
percentIbc = np.str(np.int(np.round(100*(IbcCounter/knownCounter)))) + "%"

print(percentII, percentIa, percentIbc)

types = ["Type II", "Type Ia", "Type Ib/c"]
typePercents = [percentII, percentIa, percentIbc]


typeStats = Table({'Types'    : types,
              'Percent'  : typePercents},
             names=['Types', 'Percent'])

ascii.write(typeStats, '../Data/3.TypeStats.txt', Writer = ascii.Latex,
            latexdict = {'tabletype': 'table*'})

66 38 13 8 7
64% 22% 14%
